In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/20 (46).jpg
/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/28 (25).jpg
/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/24 (36).jpg
/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/11 (63).jpg
/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/32 (63).jpg
/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/20 (39).jpg
/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/32 (65).jpg
/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/32 (32).jpg
/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/9 (77).jpg
/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/14 

In [4]:
import os

# Get the current working directory
current_dir = os.getcwd()
print("Starting Directory:", current_dir)

# Walk through the directory and display its structure
for root, dirs, files in os.walk(current_dir):
    print("\nDirectory:", root)
    print("Subdirectories:", dirs)
    print("Files:", files)

Starting Directory: /kaggle/working

Directory: /kaggle/working
Subdirectories: ['.virtual_documents']
Files: []

Directory: /kaggle/working/.virtual_documents
Subdirectories: []
Files: []


In [5]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [13]:
import torch
import torchvision
from torchvision import datasets, transforms
import os

# Define paths to your dataset
data_dir = "/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

# Define transformations for training and testing
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.RandomRotation(10),
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Check class names
class_names = train_dataset.classes
print("Classes:", class_names)

Classes: ['Mild Impairment', 'Moderate Impairment', 'No Impairment', 'Very Mild Impairment']


In [14]:
import torchvision.models as models
import torch.nn as nn

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load pre-trained ResNet-50
model = models.resnet50(pretrained=True)

# Modify the final layer for your number of classes
num_classes = len(class_names)  # e.g., 2 for binary classification
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to GPU
model = model.to(device)

Using device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
# Define loss function (with optional class weights for imbalance)
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [16]:
from sklearn.metrics import precision_score
import numpy as np

# Training loop
num_epochs = 20
best_precision = 0.0
best_model_path = "best_alzheimer_model.pth"

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Clear gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}")

    # Evaluation phase
    model.eval()
    all_preds = []
    all_labels = []
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(test_loader)
    precision = precision_score(all_labels, all_preds, average='weighted')
    print(f"Validation Loss: {avg_val_loss:.4f}, Precision: {precision:.4f}")

    # Update learning rate scheduler
    scheduler.step(avg_val_loss)

    # Save best model based on precision
    if precision > best_precision:
        best_precision = precision
        torch.save(model.state_dict(), best_model_path)
        print(f"Saved best model with precision: {best_precision:.4f}")

Epoch 1/20, Training Loss: 0.6744


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Loss: 1.8971, Precision: 0.5728
Saved best model with precision: 0.5728
Epoch 2/20, Training Loss: 0.5203
Validation Loss: 26.0998, Precision: 0.1227


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 3/20, Training Loss: 0.4666
Validation Loss: 1.2194, Precision: 0.6489
Saved best model with precision: 0.6489
Epoch 4/20, Training Loss: 0.4425
Validation Loss: 0.8393, Precision: 0.6469
Epoch 5/20, Training Loss: 0.4228
Validation Loss: 0.9177, Precision: 0.6814
Saved best model with precision: 0.6814
Epoch 6/20, Training Loss: 0.3927
Validation Loss: 66.4185, Precision: 0.1227


...

Epoch 13/20, Training Loss: 0.2572
Validation Loss: 0.8183, Precision: 0.7791
Saved best model with precision: 0.7791
Epoch 14/20, Training Loss: 0.2476
Validation Loss: 35.5493, Precision: 0.1227


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 15/20, Training Loss: 0.2187
Validation Loss: 13.2100, Precision: 0.2290


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 16/20, Training Loss: 0.1315
Validation Loss: 0.2741, Precision: 0.8946
Saved best model with precision: 0.8946
Epoch 17/20, Training Loss: 0.0981
Validation Loss: 0.5688, Precision: 0.8566
Epoch 18/20, Training Loss: 0.0777
Validation Loss: 0.2161, Precision: 0.9130
Saved best model with precision: 0.9130
Epoch 19/20, Training Loss: 0.0732
Validation Loss: 0.2087, Precision: 0.9192
Saved best model with precision: 0.9192
Epoch 20/20, Training Loss: 0.0631
Validation Loss: 0.1701, Precision: 0.9280
Saved best model with precision: 0.9280


In [21]:
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the trained ResNet-50 model
model = models.resnet50(pretrained=False)  # No pretrained weights, we'll load our own
num_classes = 4  # Based on your dataset: Mild, Moderate, No, Very Mild Impairment
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)  # Adjust final layer
model = model.to(device)

# Load the saved weights
model.load_state_dict(torch.load("best_alzheimer_model.pth", map_location=device))
model.eval()

# Image preprocessing (matching your training transforms)
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Match your training input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
])

# Function to predict impairment
def predict_impairment(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0).to(device)  # Add batch dimension

    # Make prediction
    with torch.no_grad():
        output = model(input_batch)
        _, predicted = torch.max(output, 1)
        probability = torch.nn.functional.softmax(output, dim=1)[0]

    # Class names from your dataset
    classes = ['Mild Impairment', 'Moderate Impairment', 'No Impairment', 'Very Mild Impairment']
    prediction = classes[predicted.item()]
    confidence = probability[predicted.item()].item()

    return prediction, confidence

# Example usage
if __name__ == "__main__":
    # Use an example image path from your dataset
    default_image_path = "/kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/20 (46).jpg"
    image_path = input(f"Enter the path to your image (or press Enter for default: {default_image_path}): ") or default_image_path
    try:
        prediction, confidence = predict_impairment(image_path)
        print(f"Prediction: {prediction}, Confidence: {confidence:.4f}")
    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
    except Exception as e:
        print(f"Error: {str(e)}")

<ipython-input-21-e3e4b3966483>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_alzheimer_model.pth", map_location=device))


Enter the path to your image (or press Enter for default: /kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/No Impairment/20 (46).jpg):  /kaggle/input/best-alzheimer-mri-dataset-99-accuracy/Combined Dataset/test/Mild Impairment/10 (2).jpg


Prediction: Mild Impairment, Confidence: 0.9939
